
# A)

In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

in_data = loadmat('face_emotion_data.mat')
x = in_data['X']
y = in_data['y']

w_opt = np.linalg.inv(x.T@x)@x.T@y
print("Least Squares Classifier Weights:\n",w_opt)
y_hat = np.sign(x@w_opt)



Least Squares Classifier Weights:
 [[ 0.94366942]
 [ 0.21373778]
 [ 0.26641775]
 [-0.39221373]
 [-0.00538552]
 [-0.01764687]
 [-0.16632809]
 [-0.0822838 ]
 [-0.16644364]]


# B)

The $\hat{w}$ vector indicates the weight that is to be placed on a certain distance measurement from the $X$ matrix. Since $X$ is m-by-n, each row of $X$ indicates a specific individual's measurements, and each column corresponds to a specific type of measurement from each individual. When you get a new image, take those weights and multiply them by the corresponding distance measurement and sum the results (as in, inner product $x^{T}\hat{w}$). The sign of the result will tell you whether the person is sad or happy. 

# C)

$w_1=0.94366942$ is the most significant because it has the largest magnitude - more than double the other weights. The second most significant would be $w_4=-0.39221373$ and the third is $w_3=0.26641775$.

# D)

We should choose the three most significant weights from the previous question because these have the most impact on whether we determine a face is happy or sad. 

To design the classifier, we want only features 1, 3, and 4, so we can re-train the classifier to get a new weight vector $\hat{w}$ by isolating the specific mesasurements (columns of $X$) that correspond to those weights. From there, you can calculate the classifier $\hat{y}=X\hat{w}$ and use the sign of the result to classify the emotion.

In [2]:
x_partition = x[:, [0, 2, 3]]
print("Dimensions of partitioned X:\n",x_partition.shape)

w_opt_2 = np.linalg.inv(x_partition.T@x_partition)@x_partition.T@y

print("Least Squares Classifier Weights for partitioned X:\n", w_opt_2)

y_hat_2 = np.sign(x_partition@w_opt_2)

Dimensions of partitioned X:
 (128, 3)
Least Squares Classifier Weights for partitioned X:
 [[ 0.70546316]
 [ 0.8737872 ]
 [-0.78805643]]


# E)

In [3]:
error_vec = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat, y))]
print("Errors (9 features): ", sum(error_vec))
print("%Error (9 features): ", 100*sum(error_vec)/128)

error_vec_2 = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat_2, y))]
print("Errors (3 features): ", sum(error_vec_2))
print("%Error (3 features): ", 100*sum(error_vec_2)/128)

Errors (9 features):  3
%Error (9 features):  2.34375
Errors (3 features):  8
%Error (3 features):  6.25


# F)

In [4]:
def crossValidate(x_hold, y_hold, x_train, y_train):
    ## Compute least squares estimate
    w_opt = np.linalg.inv(x_train.T@x_train)@x_train.T@y_train
    ## Compute estimated labels
    y_hat = np.sign(x_hold@w_opt)
    ## Compare holdout to trained results
    error_vec = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat, y_hold))]
    print("Errors: ", sum(error_vec))
    print("%Error: ", 100*sum(error_vec)/16)
    return sum(error_vec)

error = 0
for i in range (0, 128, 16):
    x_hold = x[i:i+16,:]
    y_hold = y[i:i+16,:]
    x_train = np.concatenate( (x[:i,:], x[i+16:,:]) )
    y_train = np.concatenate( (y[:i,:], y[i+16:,:]) )
    print("\nRange: " + str(i+1) + "-" + str(i+16))
    error = error + crossValidate(x_hold, y_hold, x_train, y_train)
print("\nTotal number of errors: ", error)
print("Error rate: ", (error/16)/8)
print("%Error: ", 100*(error/16)/8)


Range: 1-16
Errors:  1
%Error:  6.25

Range: 17-32
Errors:  1
%Error:  6.25

Range: 33-48
Errors:  2
%Error:  12.5

Range: 49-64
Errors:  1
%Error:  6.25

Range: 65-80
Errors:  0
%Error:  0.0

Range: 81-96
Errors:  1
%Error:  6.25

Range: 97-112
Errors:  0
%Error:  0.0

Range: 113-128
Errors:  0
%Error:  0.0

Total number of errors:  6
Error rate:  0.046875
%Error:  4.6875
